<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# How to Deploy a Trained FastPitch and HiFi-GAN Model using NVIDIA Riva

[Train Adapt Optimize (TAO) Toolkit](https://developer.nvidia.com/tao-toolkit) provides the capability to export your model in a format that can then be deployed using [NVIDIA Riva](https://developer.nvidia.com/riva), a highly performant application framework for multi-modal conversational AI services using GPUs. 

This tutorial explores taking two `.riva` models, the result of `tao spectro_gen` and `tao vocoder` commands, and leveraging the Riva ServiceMaker framework to aggregate all the necessary artifacts for Riva deployment to a target environment. After the models are deployed in Riva, you can issue inference requests to the server. We will demonstrate how quick and straightforward this process is. 

---
## Learning Objectives
In this notebook, you will learn how to:  
- Use Riva ServiceMaker to take a TAO exported .riva and convert it to .rmir
- Deploy the model(s) locally  on the Riva Server
- Send inference requests from a demo client using Riva API bindings.

---
## Prerequisites

Before we get started, ensure you have:
- access to NVIDIA NGC and are able to download the Riva Quick Start [resources](https://ngc.nvidia.com/catalog/resources/nvidia:riva:riva_quickstart).
- a `.riva` model file that you want to deploy. You can obtain this from `tao <task> export` (with `export_format=RIVA`). For more information on training and exporting a `.riva` model, refer to the [Speech Synthesis using TAO Toolkit].

---
## Riva ServiceMaker
Servicemaker is the set of tools that aggregates all the necessary artifacts (models, files, configurations, and user settings)
for Riva deployment to a target environment. It has two main components:

* `riva-build`
* `riva-deploy`

### Riva-build

This step helps build a Riva-ready version of the model. It’s only output is an intermediate format (called a RMIR)
of an end to end pipeline for the supported services within Riva. We are taking 2 TTS models in consideration -

* [FastPitch](https://ngc.nvidia.com/catalog/models/nvidia:tao:speechsynthesis_english_fastpitch) (spectrogram generator)
* [HiFi-GAN](https://ngc.nvidia.com/catalog/models/nvidia:tao:speechsynthesis_hifigan) (vocoder).<br>

`riva-build` is responsible for the combination of one or more exported models (.riva files) into a single file
containing an intermediate format called Riva Model Intermediate Representation (.rmir). This file contains a
deployment-agnostic specification of the whole end-to-end pipeline along with all the assets required for the
final deployment and inference. Please checkout the [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/service-tts.html#fastpitch-and-hifi-gan-pipeline-configuration) to find out more.

### Riva-deploy

The deployment tool takes as input one or more Riva Model Intermediate Representation (RMIR) files and
a target model repository directory. It creates an ensemble configuration specifying the pipeline for
the execution and finally writes all those assets to the output model repository directory.

For the purpose of this notebook, we will only be using the `riva-build` component.

In [1]:
# IMPORTANT: UPDATE THESE PATHS 

# ServiceMaker Docker
RIVA_SM_CONTAINER = "<add container name>"

# Directory where the .riva models are stored $MODEL_LOC/*.riva
# Both the FastPitch_22k_LJS.riva and HifiGAN_22k_LJS.riva models should be present
MODEL_LOC = "<add path to model location>"

# Name of the .riva file
SPECTRO_GEN_MODEL_NAME = "<add model name>"
VOCODER_MODEL_NAME = "<add model name>"

# Key that model is encrypted with, while exporting with TAO
KEY = "<add encryption key used for trained model>"

In [ ]:
# Get the ServiceMaker docker
! docker pull $RIVA_SM_CONTAINER

In [ ]:
# Syntax: riva-build <task-name> output-dir-for-rmir/model.rmir:key dir-for-riva/model.riva:key
! docker run --rm --gpus 0 -v $MODEL_LOC:/data $RIVA_SM_CONTAINER -- \
            riva-build speech_synthesis /data/tts.rmir:$KEY /data/$SPECTRO_GEN_MODEL_NAME:$KEY /data/$VOCODER_MODEL_NAME:$KEY

In [ ]:
# For a multi-speaker model, please un-comment the command below and run the following command.
# ! docker run --rm --gpus 0 -v $MODEL_LOC:/data $RIVA_SM_CONTAINER \
#              riva-build speech_synthesis /riva_models/new_speaker.rmir:tlt_encode \
#              /data/$SPECTRO_GEN_MODEL_NAME:$KEY \
#              /data/$VOCODER_MODEL_NAME:$KEY \
#              --voice_name=new_speaker \
#              --subvoices=ljspeech:0,new_voice:1

---
## Start Riva Server
Once the model repository is generated, we are ready to start the Riva server. From this step onwards you need to download the Riva QuickStart Resource from NGC. 
Set the path to the directory here:

In [ ]:
# Set the Riva QuickStart directory
RIVA_DIR = "<Path to the uncompressed folder downloaded from quickstart(include the folder name)>"

Next, we modify config.sh to enable relevant Riva services (tts in this case for fastpitch/hifigan), provide the encryption key, and path to the model repository (`riva_model_loc`) generated in the previous step among other configurations. 

For instance, if above the model repository is generated at `$MODEL_LOC/models`, then you can specify `riva_model_loc` as the same directory as `MODEL_LOC` <br>

Pretrained versions of models specified in models_asr/nlp/tts are fetched from NGC. Since we are using our custom model, we can comment it in models_tts (and any others that are not relevant to your use case). <br>

#### config.sh snippet
```
# Enable or Disable Riva Services 
service_enabled_asr=false                                                      ## MAKE CHANGES HERE
service_enabled_nlp=false                                                      ## MAKE CHANGES HERE
service_enabled_tts=true                                                     ## MAKE CHANGES HERE

# Specify one or more GPUs to use
# specifying more than one GPU is currently an experimental feature, and may result in undefined behaviours.
gpus_to_use="device=0"

# Specify the encryption key to use to deploy models
MODEL_DEPLOY_KEY="tlt_encode"                                                  ## MAKE CHANGES HERE

# Locations to use for storing models artifacts
#
# If an absolute path is specified, the data will be written to that location
# Otherwise, a docker volume will be used (default).
#
# riva_init.sh will create a `rmir` and `models` directory in the volume or
# path specified. 
#
# RMIR ($riva_model_loc/rmir)
# Riva uses an intermediate representation (RMIR) for models
# that are ready to deploy but not yet fully optimized for deployment. Pretrained
# versions can be obtained from NGC (by specifying NGC models below) and will be
# downloaded to $riva_model_loc/rmir by `riva_init.sh`
# 
# Custom models produced by NeMo or TAO and prepared using riva-build
# may also be copied manually to this location $(riva_model_loc/rmir).
#
# Models ($riva_model_loc/models)
# During the riva_init process, the RMIR files in $riva_model_loc/rmir
# are inspected and optimized for deployment. The optimized versions are
# stored in $riva_model_loc/models. The riva server exclusively uses these
# optimized versions.
riva_model_loc="<add path>"                              ## MAKE CHANGES HERE (Replace with MODEL_LOC)                      
```

In [ ]:
# Ensure you have permission to execute these scripts
! cd $RIVA_DIR && chmod +x ./riva_init.sh && chmod +x ./riva_start.sh

In [ ]:
# Run Riva Init. This will fetch the containers/models
# YOU CAN SKIP THIS STEP IF YOU DID RIVA DEPLOY
! cd $RIVA_DIR && ./riva_init.sh config.sh

In [ ]:
# Run Riva Start. This will deploy your model(s).
! cd $RIVA_DIR && ./riva_start.sh config.sh

---
## Run Inference
Once the Riva server is up and running with your models, you can send inference requests querying the server. 

To send GRPC requests, you can install Riva Python API bindings for client. This is available as a pip .whl with the QuickStart.


In [ ]:
# Install client API bindings
! cd $RIVA_DIR && pip install <add .whl file>

### Connect to Riva server and run inference
Now we actually query the Riva server, let's get started. The following cell queries the riva server(using grpc) to yield a result.

In [ ]:
import argparse
import grpc
import time
import riva_api.riva_tts_pb2 as rtts
import riva_api.riva_tts_pb2_grpc as rtts_srv
import riva_api.riva_audio_pb2 as ra
import wave
import IPython.display as ipd
import numpy as np

server = "localhost:50051"

channel = grpc.insecure_channel(server)
client = rtts_srv.RivaSpeechSynthesisStub(channel)

req = rtts.SynthesizeSpeechRequest()
req.text = "Is it recognize speech or wreck a nice beach?"
req.language_code = "en-US"                    # currently required to be "en-US"
req.encoding = ra.AudioEncoding.LINEAR_PCM     # Supports LINEAR_PCM, FLAC, MULAW and ALAW audio encodings
req.sample_rate_hz = 22050                     # ignored, audio returned will be 22.05KHz
req.voice_name = "ljspeech"                    # ignored

# For a multispeaker model, please set uncomment the line below:
# req.voice_name = "new_speaker.new_voice"

resp = client.Synthesize(req)
audio_samples = np.frombuffer(resp.audio, dtype=np.float32)
ipd.Audio(audio_samples, rate=22050)

You can stop all docker container before shutting down the jupyter kernel. **Caution: The following command will stop all running containers**

In [ ]:
! docker stop $(docker ps -a -q)